In [ ]:
!pip install --upgrade scikit-learn==0.21.2
!pip install --upgrade scikit-optimize==0.7.4
!pip install imblearn


     |████████████████████████████████| 6.7 MB 12.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.21.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.21.2 which is incompatible.
     |████████████████████████████████| 80 kB 4.9 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

seed = 7
np.random.seed(seed)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/Major Project/New/mangalath-failure modes/Shear-Wall-Failure-Mode-master/Shear_Wall_Database.xlsx'

In [ ]:
def dataprocessor(path):
  data1=pd.read_excel(path,sheet_name='Database')
  data=data1.loc[:, 'FailureMode':'Ab/Ag']
  data_mod_lc = data.copy()
  data_mod_lc['Section'] = data_mod_lc['Section'].astype('category')
  #print(cat_bird_lc.dtypes)
  data_mod_lc['Section'] = data_mod_lc['Section'].cat.codes
  data['Section'] = data_mod_lc['Section']
  data = pd.get_dummies(data = data , columns=['Section'] )
  X = data.loc[:, data.columns != 'FailureMode']
  y=data['FailureMode']-1
  return X,y




In [ ]:
# data=data1.loc[:, 'FailureMode':'Ab/Ag']
# data_mod_lc = data.copy()
# data_mod_lc['Section'] = data_mod_lc['Section'].astype('category')
# #print(cat_bird_lc.dtypes)
# data_mod_lc['Section'] = data_mod_lc['Section'].cat.codes
# data['Section'] = data_mod_lc['Section']

# c = data_mod_lc.Section.astype('category')
# d = dict(enumerate(c.cat.categories))
# print (d)

# print(data.isnull().sum())

# data = pd.get_dummies(data = data , columns=['Section'] )

# X = data.loc[:, data.columns != 'FailureMode']
# y=data['FailureMode']-1

X,y=dataprocessor(path)

In [ ]:
print(X.shape,y.shape)

(393, 11) (393,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.shape,X_test.shape)

(275, 11) (118, 11)


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'adam',neurons=64,batch_size=32,epochs=100,activation='relu',patience=3,loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(4,activation='softmax'))  # 10 is the number of classes in the dataset, you can change it based on your dataset
    model.compile(optimizer = optimizer, loss=loss,metrics=["accuracy"])
    # early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              # callbacks = [early_stopping],
              verbose=1) #verbose set to 1 will show the training process
    return model

In [ ]:
clf = KerasClassifier(build_fn=ANN, verbose=0)
scores = cross_val_score(clf, X_train, y_train, cv=3,scoring='accuracy')
print(scores)
print("Accuracy:"+ str(scores.mean()))

Epoch 1/100
13/13 [==============================] - 1s 2ms/step - loss: 1.4348 - accuracy: 0.3130
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 1.1736 - accuracy: 0.5725
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 1.0804 - accuracy: 0.5496
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 1.0221 - accuracy: 0.5751
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.9912 - accuracy: 0.6285
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 0.9714 - accuracy: 0.6260
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.9413 - accuracy: 0.6514
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 0.9295 - accuracy: 0.6718
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.9240 - accuracy: 0.6539
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.9109 - accuracy: 0.6845
Epoch 11/

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

def ANN(optimizer = 'adam',neurons=64,batch_size=32,epochs=500,activation='relu',patience=3,loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(4,activation='softmax'))  # 10 is the number of classes in the dataset, you can change it based on your dataset
    model.compile(optimizer = optimizer, loss=loss,metrics=["accuracy"])
    # early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              # callbacks = [early_stopping],
              verbose=1) #verbose set to 1 will show the training process
    return model


In [ ]:
# model=ANN()
# history=model.fit(X_train,pd.get_dummies(y_train).values,validation_data=(X_test,pd.get_dummies(y_test).values),epochs=100)

In [ ]:
# model.evaluate(X_test,pd.get_dummies(y_test))

In [ ]:
# print(history.history.keys())
# plt.figure()
# plt.title('model accuracy')
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# plt.figure()
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper right')
# plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32],
    'neurons':[16,32],
    'epochs':[20,50],
    'patience':[2,5]
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, rf_params, cv=3,scoring='accuracy')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(grid.best_score_))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

25/25 [==============================] - 1s 2ms/step - loss: 1.2689 - accuracy: 0.4046
Epoch 2/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1590 - accuracy: 0.5344
Epoch 3/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1059 - accuracy: 0.6081
Epoch 4/20
25/25 [==============================] - 0s 2ms/step - loss: 1.0658 - accuracy: 0.6031
Epoch 5/20
25/25 [==============================] - 0s 2ms/step - loss: 1.0291 - accuracy: 0.6361
Epoch 6/20
25/25 [==============================] - 0s 2ms/step - loss: 1.0113 - accuracy: 0.6387
Epoch 7/20
25/25 [==============================] - 0s 2ms/step - loss: 0.9857 - accuracy: 0.6768
Epoch 8/20
25/25 [==============================] - 0s 2ms/step - loss: 0.9710 - accuracy: 0.6514
Epoch 9/20
25/25 [==============================] - 0s 2ms/step - loss: 0.9648 - accuracy: 0.6870
Epoch 10/20
25/25 [==============================] - 0s 2ms/step - loss: 0.9672 - accuracy: 0.6361
Epoch 11/20
25/25 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 1s 2ms/step - loss: 2.5547 - accuracy: 0.2697
Epoch 2/20
25/25 [==============================] - 0s 2ms/step - loss: 1.5671 - accuracy: 0.3511
Epoch 3/20
25/25 [==============================] - 0s 2ms/step - loss: 1.3756 - accuracy: 0.5089
Epoch 4/20
25/25 [==============================] - 0s 2ms/step - loss: 1.2697 - accuracy: 0.5318
Epoch 5/20
25/25 [==============================] - 0s 2ms/step - loss: 1.2141 - accuracy: 0.5165
Epoch 6/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1828 - accuracy: 0.5547
Epoch 7/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1562 - accuracy: 0.5852
Epoch 8/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1348 - accuracy: 0.5980
Epoch 9/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1061 - accuracy: 0.5903
Epoch 10/20
25/25 [==============================] - 0s 2ms/step - loss: 1.0764 - accuracy: 0.5903
Epoch 11/20
25/25 [===========

...

13/13 [==============================] - 0s 3ms/step - loss: 1.2830 - accuracy: 0.3333
Epoch 2/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1985 - accuracy: 0.4885
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 1.1843 - accuracy: 0.5216
Epoch 4/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1713 - accuracy: 0.5064
Epoch 5/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1592 - accuracy: 0.5242
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1470 - accuracy: 0.5471
Epoch 7/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1349 - accuracy: 0.5471
Epoch 8/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1291 - accuracy: 0.5496
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 1.1107 - accuracy: 0.5623
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 1.1008 - accuracy: 0.5700
Epoch 11/50
13/13 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


13/13 [==============================] - 0s 3ms/step - loss: 1.4125 - accuracy: 0.1578
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2912 - accuracy: 0.2366
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2623 - accuracy: 0.3435
Epoch 4/50
13/13 [==============================] - 0s 2ms/step - loss: 1.2463 - accuracy: 0.3715
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2355 - accuracy: 0.4173
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2310 - accuracy: 0.3969
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2259 - accuracy: 0.4020
Epoch 8/50
13/13 [==============================] - 0s 2ms/step - loss: 1.2230 - accuracy: 0.3817
Epoch 9/50
13/13 [==============================] - 0s 2ms/step - loss: 1.2168 - accuracy: 0.4300
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 1.2144 - accuracy: 0.4198
Epoch 11/50
13/13 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:793: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 0s 2ms/step - loss: 1.3663 - accuracy: 0.3206
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 1.2585 - accuracy: 0.4784
Epoch 3/20
25/25 [==============================] - 0s 2ms/step - loss: 1.2116 - accuracy: 0.5344
Epoch 4/20
25/25 [==============================] - 0s 2ms/step - loss: 1.1762 - accuracy: 0.5496
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 1.1467 - accuracy: 0.5496
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 1.1182 - accuracy: 0.5547
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 1.0950 - accuracy: 0.5802
Epoch 8/20
25/25 [==============================] - 0s 3ms/step - loss: 1.0668 - accuracy: 0.5827
Epoch 9/20
25/25 [==============================] - 0s 4ms/step - loss: 1.0563 - accuracy: 0.5802
Epoch 10/20
25/25 [==============================] - 0s 2ms/step - loss: 1.0315 - accuracy: 0.5751
Epoch 11/20
25/25 [===========

In [ ]:
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':sp_randint(3,20)
}
n_iter_search=10
clf = KerasClassifier(build_fn=ANN, verbose=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='accuracy')
Random.fit(X, y)
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informatio

25/25 [==============================] - 1s 3ms/step - loss: 1.3271 - accuracy: 0.3791
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 1.1000 - accuracy: 0.5496
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0214 - accuracy: 0.5903
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0149 - accuracy: 0.5827
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9493 - accuracy: 0.6539
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.8938 - accuracy: 0.6667
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9024 - accuracy: 0.6438
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8754 - accuracy: 0.6641
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8465 - accuracy: 0.6616
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8549 - accuracy: 0.6667
Epoch 11/50
25/25 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 1s 2ms/step - loss: 1.1505 - accuracy: 0.4860
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 1.0227 - accuracy: 0.5878
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9727 - accuracy: 0.6183
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9279 - accuracy: 0.6361
Epoch 5/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9192 - accuracy: 0.6463
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8863 - accuracy: 0.6616
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8813 - accuracy: 0.6641
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8625 - accuracy: 0.6794
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8312 - accuracy: 0.7074
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.8441 - accuracy: 0.6718
Epoch 11/50
25/25 [===========

...

25/25 [==============================] - 1s 2ms/step - loss: 1.3497 - accuracy: 0.3791
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 1.2052 - accuracy: 0.4707
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 1.1468 - accuracy: 0.5064
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0897 - accuracy: 0.5827
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0383 - accuracy: 0.5878
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9983 - accuracy: 0.6005
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9698 - accuracy: 0.6234
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9498 - accuracy: 0.6158
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9335 - accuracy: 0.6387
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9089 - accuracy: 0.6768
Epoch 11/50
25/25 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 1s 2ms/step - loss: 1.3078 - accuracy: 0.3333
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 1.2344 - accuracy: 0.3868
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 1.2174 - accuracy: 0.3969
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 1.1907 - accuracy: 0.4555
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 1.1403 - accuracy: 0.5700
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0827 - accuracy: 0.5852
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0435 - accuracy: 0.5852
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 1.0214 - accuracy: 0.5776
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.9981 - accuracy: 0.6056
Epoch 10/50
25/25 [==============================] - 0s 3ms/step - loss: 0.9681 - accuracy: 0.6285
Epoch 11/50
25/25 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:793: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


13/13 [==============================] - 1s 2ms/step - loss: 1.2119 - accuracy: 0.4758
Epoch 2/20
13/13 [==============================] - 0s 2ms/step - loss: 1.1320 - accuracy: 0.5522
Epoch 3/20
13/13 [==============================] - 0s 2ms/step - loss: 1.0860 - accuracy: 0.5674
Epoch 4/20
13/13 [==============================] - 0s 2ms/step - loss: 1.0451 - accuracy: 0.5852
Epoch 5/20
13/13 [==============================] - 0s 2ms/step - loss: 1.0217 - accuracy: 0.5751
Epoch 6/20
13/13 [==============================] - 0s 2ms/step - loss: 1.0083 - accuracy: 0.6209
Epoch 7/20
13/13 [==============================] - 0s 3ms/step - loss: 0.9803 - accuracy: 0.6463
Epoch 8/20
13/13 [==============================] - 0s 3ms/step - loss: 0.9599 - accuracy: 0.6514
Epoch 9/20
13/13 [==============================] - 0s 2ms/step - loss: 0.9461 - accuracy: 0.6438
Epoch 10/20
13/13 [==============================] - 0s 2ms/step - loss: 0.9280 - accuracy: 0.6489
Epoch 11/20
13/13 [===========

In [ ]:
!git clone https://github.com/thuijskens/scikit-hyperband.git

Cloning into 'scikit-hyperband'...
remote: Enumerating objects: 408, done.
remote: Total 408 (delta 0), reused 0 (delta 0), pack-reused 408
Receiving objects: 100% (408/408), 5.55 MiB | 29.32 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [ ]:
%cd "/content/scikit-hyperband"
!ls

/content/scikit-hyperband
circle.yml  doc       hyperband  MANIFEST.in  requirements.txt	setup.py
ci_scripts  examples  LICENSE	 README.md    setup.cfg


In [ ]:
!python setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating scikit_hyperband.egg-info
writing scikit_hyperband.egg-info/PKG-INFO
writing dependency_links to scikit_hyperband.egg-info/dependency_links.txt
writing requirements to scikit_hyperband.egg-info/requires.txt
writing top-level names to scikit_hyperband.egg-info/top_level.txt
writing manifest file 'scikit_hyperband.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'scikit_hyperband.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/hyperband
copying hyperband/search.py -> build/lib/hyperband
copying hyperband/_

In [ ]:
# ANN
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':sp_randint(3,20)
}
clf = KerasClassifier(build_fn=ANN, epochs=20, verbose=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=10,scoring='accuracy',resource_param='epochs')
hyper.fit(X, y)
print(hyper.best_params_)
print("Accuracy:"+ str(hyper.best_score_))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additi

25/25 [==============================] - 1s 3ms/step - loss: 1.1356 - accuracy: 0.4885


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 1s 2ms/step - loss: 1.2153 - accuracy: 0.4707


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


...

13/13 [==============================] - 1s 2ms/step - loss: 1.2623 - accuracy: 0.4275
Epoch 2/10
13/13 [==============================] - 0s 3ms/step - loss: 1.1060 - accuracy: 0.5267
Epoch 3/10
13/13 [==============================] - 0s 2ms/step - loss: 1.0712 - accuracy: 0.5700
Epoch 4/10
13/13 [==============================] - 0s 3ms/step - loss: 1.0307 - accuracy: 0.5903
Epoch 5/10
13/13 [==============================] - 0s 3ms/step - loss: 0.9921 - accuracy: 0.5852
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 0.9892 - accuracy: 0.5954
Epoch 7/10
13/13 [==============================] - 0s 3ms/step - loss: 0.9616 - accuracy: 0.6183
Epoch 8/10
13/13 [==============================] - 0s 3ms/step - loss: 0.9335 - accuracy: 0.6387
Epoch 9/10
13/13 [==============================] - 0s 3ms/step - loss: 0.8992 - accuracy: 0.6692
Epoch 10/10
13/13 [==============================] - 0s 3ms/step - loss: 0.9453 - accuracy: 0.6132
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


13/13 [==============================] - 1s 2ms/step - loss: 1.1992 - accuracy: 0.4580
Epoch 2/10
13/13 [==============================] - 0s 3ms/step - loss: 1.0623 - accuracy: 0.5318
Epoch 3/10
13/13 [==============================] - 0s 2ms/step - loss: 1.0598 - accuracy: 0.5623
Epoch 4/10
13/13 [==============================] - 0s 2ms/step - loss: 1.0016 - accuracy: 0.6107
Epoch 5/10
13/13 [==============================] - 0s 2ms/step - loss: 0.9738 - accuracy: 0.6183
Epoch 6/10
13/13 [==============================] - 0s 3ms/step - loss: 0.9817 - accuracy: 0.6081
Epoch 7/10
13/13 [==============================] - 0s 2ms/step - loss: 0.9444 - accuracy: 0.6438
Epoch 8/10
13/13 [==============================] - 0s 2ms/step - loss: 0.9443 - accuracy: 0.6285
Epoch 9/10
13/13 [==============================] - 0s 2ms/step - loss: 0.9279 - accuracy: 0.6310
Epoch 10/10
13/13 [==============================] - 0s 2ms/step - loss: 0.8956 - accuracy: 0.6539
Epoch 1/3


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:793: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


25/25 [==============================] - 1s 2ms/step - loss: 1.1709 - accuracy: 0.4835
Epoch 2/3
25/25 [==============================] - 0s 2ms/step - loss: 1.0560 - accuracy: 0.5445
Epoch 3/3
25/25 [==============================] - 0s 2ms/step - loss: 0.9866 - accuracy: 0.5929
{'activation': 'tanh', 'batch_size': 16, 'epochs': 3, 'neurons': 64, 'optimizer': 'rmsprop', 'patience': 10}
Accuracy:0.5903307888040712


In [ ]:
#ANN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':Integer(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':Integer(3,20)
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='accuracy')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("Accuracy:"+ str(Bayes.best_score_))

Epoch 1/23


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informatio

7/7 [==============================] - 1s 3ms/step - loss: 1.4929 - accuracy: 0.3104
Epoch 2/23
7/7 [==============================] - 0s 3ms/step - loss: 1.3188 - accuracy: 0.3359
Epoch 3/23
7/7 [==============================] - 0s 3ms/step - loss: 1.2459 - accuracy: 0.4478
Epoch 4/23
7/7 [==============================] - 0s 3ms/step - loss: 1.2331 - accuracy: 0.5344
Epoch 5/23
7/7 [==============================] - 0s 3ms/step - loss: 1.2022 - accuracy: 0.5344
Epoch 6/23
7/7 [==============================] - 0s 3ms/step - loss: 1.1641 - accuracy: 0.5369
Epoch 7/23
7/7 [==============================] - 0s 3ms/step - loss: 1.1369 - accuracy: 0.5852
Epoch 8/23
7/7 [==============================] - 0s 4ms/step - loss: 1.1066 - accuracy: 0.5776
Epoch 9/23
7/7 [==============================] - 0s 3ms/step - loss: 1.0783 - accuracy: 0.5929
Epoch 10/23
7/7 [==============================] - 0s 3ms/step - loss: 1.0528 - accuracy: 0.5980
Epoch 11/23
7/7 [==============================] -

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


7/7 [==============================] - 1s 4ms/step - loss: 1.4155 - accuracy: 0.3868
Epoch 2/23
7/7 [==============================] - 0s 3ms/step - loss: 1.2326 - accuracy: 0.5089
Epoch 3/23
7/7 [==============================] - 0s 3ms/step - loss: 1.1595 - accuracy: 0.5573
Epoch 4/23
7/7 [==============================] - 0s 3ms/step - loss: 1.1118 - accuracy: 0.6081
Epoch 5/23
7/7 [==============================] - 0s 2ms/step - loss: 1.0738 - accuracy: 0.5852
Epoch 6/23
7/7 [==============================] - 0s 3ms/step - loss: 1.0452 - accuracy: 0.5878
Epoch 7/23
7/7 [==============================] - 0s 3ms/step - loss: 1.0207 - accuracy: 0.6056
Epoch 8/23
7/7 [==============================] - 0s 2ms/step - loss: 1.0013 - accuracy: 0.6539
Epoch 9/23
7/7 [==============================] - 0s 3ms/step - loss: 0.9965 - accuracy: 0.6641
Epoch 10/23
7/7 [==============================] - 0s 3ms/step - loss: 0.9762 - accuracy: 0.6539
Epoch 11/23
7/7 [==============================] -

...

7/7 [==============================] - 0s 3ms/step - loss: 1.2675 - accuracy: 0.3003
Epoch 2/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2631 - accuracy: 0.3842
Epoch 3/24
7/7 [==============================] - 0s 4ms/step - loss: 1.2369 - accuracy: 0.4962
Epoch 4/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2032 - accuracy: 0.5318
Epoch 5/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1910 - accuracy: 0.5344
Epoch 6/24
7/7 [==============================] - 0s 4ms/step - loss: 1.1999 - accuracy: 0.5064
Epoch 7/24
7/7 [==============================] - 0s 4ms/step - loss: 1.2036 - accuracy: 0.5216
Epoch 8/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1579 - accuracy: 0.5852
Epoch 9/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1509 - accuracy: 0.5929
Epoch 10/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1464 - accuracy: 0.5954
Epoch 11/24
7/7 [==============================] -

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


7/7 [==============================] - 0s 4ms/step - loss: 1.6096 - accuracy: 0.3003
Epoch 2/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2551 - accuracy: 0.5267
Epoch 3/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2213 - accuracy: 0.5267
Epoch 4/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2098 - accuracy: 0.5216
Epoch 5/24
7/7 [==============================] - 0s 4ms/step - loss: 1.2180 - accuracy: 0.5064
Epoch 6/24
7/7 [==============================] - 0s 4ms/step - loss: 1.1971 - accuracy: 0.5293
Epoch 7/24
7/7 [==============================] - 0s 4ms/step - loss: 1.1626 - accuracy: 0.5598
Epoch 8/24
7/7 [==============================] - 0s 4ms/step - loss: 1.1663 - accuracy: 0.5344
Epoch 9/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1517 - accuracy: 0.5598
Epoch 10/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1287 - accuracy: 0.5547
Epoch 11/24
7/7 [==============================] -

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/label.py:749: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.array(y > threshold, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


7/7 [==============================] - 0s 3ms/step - loss: 1.3528 - accuracy: 0.3868
Epoch 2/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2945 - accuracy: 0.3868
Epoch 3/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2627 - accuracy: 0.3868
Epoch 4/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2401 - accuracy: 0.4020
Epoch 5/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2265 - accuracy: 0.5242
Epoch 6/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2149 - accuracy: 0.5344
Epoch 7/24
7/7 [==============================] - 0s 3ms/step - loss: 1.2042 - accuracy: 0.5394
Epoch 8/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1958 - accuracy: 0.5649
Epoch 9/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1885 - accuracy: 0.5878
Epoch 10/24
7/7 [==============================] - 0s 3ms/step - loss: 1.1779 - accuracy: 0.5929
Epoch 11/24
7/7 [==============================] -

In [ ]:
#ANN
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        "optimizer":str(params['optimizer']),
        "activation":str(params['activation']),
        'batch_size': abs(int(params['batch_size'])),
        'neurons': abs(int(params['neurons'])),
        'epochs': abs(int(params['epochs'])),
        'patience': abs(int(params['patience']))
    }
    clf = KerasClassifier(build_fn=ANN,**params, verbose=0)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, 
                                    scoring="accuracy"))

    return {'loss':score, 'status': STATUS_OK }

space = {
    "optimizer":hp.choice('optimizer',['adam','rmsprop','sgd']),
    "activation":hp.choice('activation',['relu','tanh']),
    'batch_size': hp.quniform('batch_size', 16, 64, 16),
    'neurons': hp.quniform('neurons', 10, 100, 10),
    'epochs': hp.quniform('epochs', 20, 50, 10),
    'patience': hp.quniform('patience', 3, 20, 3),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("ANN: Hyperopt estimated optimum {}".format(best))

Epoch 1/50

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  del sys.path[0]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:102: Depr

13/13 [==============================] - 1s 3ms/step - loss: 1.3293 - accuracy: 0.4555

Epoch 2/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1874 - accuracy: 0.5165

Epoch 3/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1576 - accuracy: 0.5038

Epoch 4/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1439 - accuracy: 0.5522

Epoch 5/50

13/13 [==============================] - 0s 4ms/step - loss: 1.1076 - accuracy: 0.5420

Epoch 6/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1093 - accuracy: 0.5522

Epoch 7/50

13/13 [==============================] - 0s 3ms/step - loss: 1.0818 - accuracy: 0.5674

Epoch 8/50

13/13 [==============================] - 0s 3ms/step - loss: 1.0648 - accuracy: 0.5802

Epoch 9/50

13/13 [==============================] - 0s 4ms/step - loss: 1.0403 - accuracy: 0.5903

Epoch 10/50

13/13 [==============================] - 0s 3ms/step - loss: 1.0429 - accuracy: 0.5776

Epoch 11/50

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)



13/13 [==============================] - 1s 3ms/step - loss: 1.3840 - accuracy: 0.3104

Epoch 2/50

13/13 [==============================] - 0s 2ms/step - loss: 1.2138 - accuracy: 0.5216

Epoch 3/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1594 - accuracy: 0.6158

Epoch 4/50

13/13 [==============================] - 0s 3ms/step - loss: 1.1081 - accuracy: 0.6081

Epoch 5/50

13/13 [==============================] - 0s 3ms/step - loss: 1.0588 - accuracy: 0.6336

Epoch 6/50

13/13 [==============================] - 0s 3ms/step - loss: 1.0264 - accuracy: 0.6361

Epoch 7/50

13/13 [==============================] - 0s 4ms/step - loss: 1.0029 - accuracy: 0.6489

Epoch 8/50

13/13 [==============================] - 0s 3ms/step - loss: 0.9889 - accuracy: 0.6514

Epoch 9/50

13/13 [==============================] - 0s 3ms/step - loss: 0.9665 - accuracy: 0.6692

Epoch 10/50

13/13 [==============================] - 0s 3ms/step - loss: 0.9682 - accuracy: 0.6718

Epoch 11/50

...

7/7 [==============================] - 1s 7ms/step - loss: 1.3598 - accuracy: 0.2901

Epoch 2/30

7/7 [==============================] - 0s 6ms/step - loss: 1.2657 - accuracy: 0.3893

Epoch 3/30

7/7 [==============================] - 0s 5ms/step - loss: 1.2406 - accuracy: 0.4631

Epoch 4/30

7/7 [==============================] - 0s 5ms/step - loss: 1.2200 - accuracy: 0.4682

Epoch 5/30

7/7 [==============================] - 0s 4ms/step - loss: 1.2010 - accuracy: 0.5038

Epoch 6/30

7/7 [==============================] - 0s 4ms/step - loss: 1.1788 - accuracy: 0.5547

Epoch 7/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1793 - accuracy: 0.4911

Epoch 8/30

7/7 [==============================] - 0s 6ms/step - loss: 1.1538 - accuracy: 0.5522

Epoch 9/30

7/7 [==============================] - 0s 8ms/step - loss: 1.1400 - accuracy: 0.5394

Epoch 10/30

7/7 [==============================] - 0s 4ms/step - loss: 1.1457 - accuracy: 0.5165

Epoch 11/30

7/7 [=============

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)



7/7 [==============================] - 0s 6ms/step - loss: 1.3461 - accuracy: 0.3944

Epoch 2/30

7/7 [==============================] - 0s 5ms/step - loss: 1.2139 - accuracy: 0.4758

Epoch 3/30

7/7 [==============================] - 0s 8ms/step - loss: 1.2096 - accuracy: 0.5038

Epoch 4/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1816 - accuracy: 0.5496

Epoch 5/30

7/7 [==============================] - 0s 6ms/step - loss: 1.1632 - accuracy: 0.5216

Epoch 6/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1507 - accuracy: 0.5165

Epoch 7/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1335 - accuracy: 0.5802

Epoch 8/30

7/7 [==============================] - 0s 6ms/step - loss: 1.1205 - accuracy: 0.5751

Epoch 9/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1136 - accuracy: 0.5674

Epoch 10/30

7/7 [==============================] - 0s 6ms/step - loss: 1.1069 - accuracy: 0.5776

Epoch 11/30

7/7 [=============

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)



7/7 [==============================] - 1s 7ms/step - loss: 1.2837 - accuracy: 0.2672

Epoch 2/30

7/7 [==============================] - 0s 6ms/step - loss: 1.2474 - accuracy: 0.3919

Epoch 3/30

7/7 [==============================] - 0s 6ms/step - loss: 1.2005 - accuracy: 0.4860

Epoch 4/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1839 - accuracy: 0.5115

Epoch 5/30

7/7 [==============================] - 0s 4ms/step - loss: 1.1570 - accuracy: 0.5165

Epoch 6/30

7/7 [==============================] - 0s 6ms/step - loss: 1.1400 - accuracy: 0.5115

Epoch 7/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1221 - accuracy: 0.5751

Epoch 8/30

7/7 [==============================] - 0s 8ms/step - loss: 1.1131 - accuracy: 0.5369

Epoch 9/30

7/7 [==============================] - 0s 7ms/step - loss: 1.1031 - accuracy: 0.5623

Epoch 10/30

7/7 [==============================] - 0s 5ms/step - loss: 1.1014 - accuracy: 0.5649

Epoch 11/30

7/7 [=============

In [ ]:
!pip install optunity

     |████████████████████████████████| 4.6 MB 8.4 MB/s 
  Created wheel for optunity: filename=Optunity-1.1.1-py3-none-any.whl size=72032 sha256=4ae8fd0a4aeb55f030fbe05ac84e48aae5bd14921cbca1306b7d3a08301a0325
  Stored in directory: /root/.cache/pip/wheels/0d/0e/31/a4763ff822a20f88a61dd3bb1c8b78a8f25c605392fe956d07
Successfully built optunity


In [ ]:
#ANN
import optunity
import optunity.metrics

data=X.values
labels=y.values.tolist()

search = {
    'optimizer':[0,3],
    'activation':[0,2],
    'batch_size': [0, 2],
    'neurons': [10, 100],
    'epochs': [20, 50],
    'patience': [3, 20],
         }
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test,optimizer=None,activation=None,batch_size=None,neurons=None,epochs=None,patience=None):
    # fit the model
    if optimizer<1:
        op='adam'
    elif optimizer<2:
        op='sgd'
    else:
        op='rmsprop'
    if activation<1:
        ac='relu'
    else:
        ac='tanh'
    if batch_size<1:
        ba=16
    else:
        ba=32
    model = ANN(optimizer=op,
                activation=ac,
                batch_size=ba,
                neurons=int(neurons),
                epochs=int(epochs),
                patience=int(patience)
                                  )
    clf = KerasClassifier(build_fn=ANN, verbose=0)
    scores=np.mean(cross_val_score(clf, X, y, cv=3, 
                                    scoring="accuracy"))

    return scores

optimal_configuration, info, _ = optunity.maximize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install sklearn-deap

     |████████████████████████████████| 24.8 MB 2.0 MB/s 
     |████████████████████████████████| 160 kB 46.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2


In [ ]:
#ANN
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':range(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':range(3,20)
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additi

Types [1, 1, 1, 1, 1, 1] and maxint [2, 1, 2, 89, 1, 16] detected
--- Evolve in 55080 possible combinations ---
Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: 1.3671 - accuracy: 0.3817
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 1.1942 - accuracy: 0.5115
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 1.1483 - accuracy: 0.5547
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 1.1381 - accuracy: 0.5598
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0828 - accuracy: 0.5725
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0891 - accuracy: 0.5522
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0372 - accuracy: 0.5852
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0471 - accuracy: 0.5700
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 1.0183 - accuracy: 0.5954
Epoch 

TypeError: ignored